In [ ]:
println("Connect Spark first")

# Make Dataframes from CSV files

In [ ]:
var US_Cases = spark.read.csv("hdfs://localhost:9000/user/hive/warehouse/time_series_covid_19_confirmed_US.csv")
var US_Deaths = spark.read.csv("hdfs://localhost:9000/user/hive/warehouse/time_series_covid_19_deaths_US.csv")

US_Deaths.createOrReplaceTempView("US_Deaths")
US_Cases.createOrReplaceTempView("US_Cases")


# Top 5 Urban areas with highest COVID deaths

In [ ]:
spark.sql("Select cast(_c478 as integer), _c10 deaths, _c11 population from US_Deaths order by 1 desc limit 5;").show

# Top 5 non Urban areas highest COVID deaths

In [ ]:
spark.sql("Select cast(_c478 as integer) deaths, _c10 city, _c11 population from US_Deaths where _c11 < 50000 and _c11 > 50 order by 1 desc limit 5;").show

# Top 5 non urban areas with Highest Percentage of COVID deaths

In [ ]:
spark.sql("select _c10 city, concat(round((  (cast(_c478 as integer) / _c11) *100       ),2),'%') as DeathPercent from US_deaths where _c11 < 50000 and _c11 >50 order by 2 desc limit 5;").show

# Top 5 urban areas with Highest Percentage of COVID deaths

In [ ]:
spark.sql("select _c10 city, concat(round(((cast(_c478 as integer) / _c11) *100 ),2),'%') as DeathPercent  from US_deaths where _c11 > 50000 order by 2 desc limit 5;").show

# Average COVID deaths in Urban area

In [ ]:
spark.sql("select round(avg(_c11),0) as avg_population, round( avg( cast(_c478 as integer)   ),0  ) avg_deaths,  concat(round((round( avg( cast(_c478 as integer)   ),0  )/round(avg(_c11),0) ) *100,2),'%')  pop_over_death_times_100                                  from US_Deaths where _c11 > 50000").show

In [ ]:
spark.sql("select _c10 city, cast(_c478 as integer) deaths from US_Deaths order by RAND() limit 5;")

# Clean the Data

In [ ]:
var US_Cases = spark.read.option("header","true").csv("hdfs://localhost:9000/user/hive/warehouse/time_series_covid_19_confirmed_US.csv")
val CC = US_Cases.select("Admin2","Combined_Key","Province_State","5/2/21")
CC.show()

In [ ]:
var US_Deaths = spark.read.option("header","true").csv("hdfs://localhost:9000/user/hive/warehouse/time_series_covid_19_deaths_US.csv")
val CD = US_Deaths.select("Admin2","Combined_Key","Population","Province_State","5/2/21")
CD.show()

# Put clean data in Parquet files

In [ ]:
CC.write.parquet("hdfs://localhost:9000/user/hive/warehouse/Cases.parquet")
CD.write.parquet("hdfs://localhost:9000/user/hive/warehouse/Deaths.parquet")
